### Sampler를 이용한 실험 결과 (전체)
- Test f1 score 기준 내림차순 정렬

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("../experimental_records.csv")
df = df[df['model_name'].apply(lambda x: True if x.find('randomForest') + 1 else False)]
df.sort_values(by='test_f1_score', ascending=False)

,date,model_name,test_f1_score,description
29,2024-02-13/23:54:57,randomForest_params05,0.717894,randomForest_params02 실험에서 ccp_alpha 값을 0.00005 만큼 감소
35,2024-02-21/22:52:54,randomForest_params05_logTransform,0.717894,randomForest_params05 실험에서 lead_desc_length feature에 log transformation 적용
26,2024-02-13/23:43:00,randomForest_params02,0.717277,randomForest_params01 실험에서 n_estimators를 100으로 증가
28,2024-02-13/23:52:17,randomForest_params04,0.710445,randomForest_params02 실험에서 ccp_alpha 값을 0.00005 만큼 높임
25,2024-02-13/23:27:34,randomForest_params01,0.704509,모든 feature 사용/전처리 X/RandomForestClassifier 사용/결측치는 0으로 채움/Seed 33/ensemble_decisiontree_w_undersampling_and_ccp 실험과 유사하게 param setting/
33,2024-02-21/14:45:37,randomForest_params05_delFeatures,0.672525,"randomForest_params05 세팅 + is_converted가 True인 데이터와 False인 데이터 간에 분포 차이가 많이 나지 않는 features를 모두 삭제하고, 7개만 사용"
34,2024-02-21/14:52:03,randomForest_params06,0.669417,randomForest_params05_delFeatrues 세팅에서 ccp_alpha 값을 0.001 감소
32,2024-02-18/22:04:47,randomForest_params05_cityAndCountry,0.647125,randomForest_params05 실험 + customer_country feature를 city와 country로 분리
27,2024-02-13/23:47:35,randomForest_params03,0.628806,"randomForest_params02 실험에서 max_features를 ""sqrt""로 변경"


---
---

**<span style='color:orange'>[0.717894, Best]</span> 실험 01: RandomForestClassifier**

**Motivation**
- Undersampling, cost-complexity pruning을 각각의 decision tree에 적용한 뒤 ensemble했을 때 test f1 score가 크게 향상된다는 사실을 이전 실험을 통해 파악하였음.
- 여러 개의 decision tree를 ensemble하는 것은 random forest 전략과 동일한데, 직접 구현한 것과 `sklearn.ensemble.RandomForestClassifier()` 를 이용하는 것 사이의 성능 차이를 확인해보고 싶었고, 본 실험을 진행하게 되었음.
- `sklearn.ensemble.RandomForestClassifier()`의 경우 다양한 hyperparameters를 지원하기 때문에, 조합에 따라 다양한 모델을 만들 수 있다는 장점이 존재함

**Basic rfc params**
```python
rfc_params_01 = {
    'n_estimators': 30, # 사용할 decision tree의 개수
    'criterion': 'gini', # statement의 분류 성능을 평가할 기준
    'max_depth': None, # tree의 최대 깊이
    'min_samples_split': 2, # internal node를 나누기 위해 필요한 최소 샘플 개수 (이 값 이하면 split X)
    'min_samples_leaf': 1, # min_samples_leaf 이상의 samples을 가져야만 leaf node로 간주됨 (?)
    'min_weight_fraction_leaf': 0.0, # ?
    'max_features': None, # best split을 찾기 위해 고려할 features의 개수
    'max_leaf_nodes': None, # ?
    'min_impurity_decrease': 0.0, # 특정 node를 split할 때, impurity가 이 값 이상 감소해야만 split을 수행
    'bootstrap': True, # sampling을 통해 만든 subset 간 데이터 중복을 허용하는지 여부. (False이면 모든 tree가 동일한 dataset 이용)
    'oob_score': True, # out-of-bag sample을 이용해 일반화 성능을 측정할 때 사용 (bootstrap==True일 때만 사용가능)
    'n_jobs': None, # 작업을 병렬적으로 수행하고 싶을 때 사용 (-1이면 모든 CPU 코어를 사용)
    'random_state': hparams['seed'], # 모델링 과정에 필요한 randomness를 부여할 때 사용할 값
    'verbose': 1, # fitting 과정에 대한 정보를 어느 정도로 출력할 지 결정
    'warm_start': False, # 이전에 만들었던 forest가 존재한다면, 해당 정보를 참고하여 새로운 forest를 build (재현가능성을 위해 False로 고정)
    'class_weight': "balanced_subsample", # class 비율을 고려하여 sampling할 때 사용
    'ccp_alpha': 0.00045, # cost-complexity pruning에 사용할 alpha 값
    'max_samples': None, # sampling 할 데이터의 총 개수
    # 'monotonic_cst': None, # available at >= 1.4
}   
```

**Test f1 score**
- `RandomForestClassifier()` 의 다양한 hyperparameters 에 따른 실험 결과 table
- Previous best test f1 score: 0.712175

    |rfc_params|test f1 score| description |
    |:-:|:-:|:-:|
    | 01 | 0.704509 (-0.007666) | Basic rfc params 참고 |
    | 02 | 0.717277 (+0.005102) | params01 에서 n_estimators를 100으로 증가 |
    | 03 | 0.628806 (-0.083369) | params02 실험에서 max_features를 "sqrt"로 변경 |
    | 04 | 0.710445 (-0.00173)  | params02 실험에서 ccp_alpha 값을 0.00005 만큼 높임 |
    | 05 | 0.717894 (+0.005719) | params02 실험에서 ccp_alpha 값을 0.00005 만큼 감소 |

**Analysis**
- `sklearn.ensemble.RandomForestClassifier()` 또한 `sklearn.tree.DecisionTreeClassifier()` 를 기반으로 구현된 것이다 보니, `02_use_sampler.ipynb` 에서 실험했던 결과와 유의미한 차이를 보이지 않았음.
- ensemble에 사용하는 tree의 개수를 100개까지 높이고, ccp_alpha 값을 조금 줄여주었을 때 test f1 score가 아주 소폭 향상되었음.

**Future works**
- gradient boosting models
---
---

**[0.717894] 실험 02: Skewed data transformation**

**Motivation**
- Feature distribution이 한 쪽으로 치우쳐져 있는 skewed feature에 대해 log transformation을 가해 모델이 극단적인 분포를 학습하지 않도록 하고자 하였음.
- 여러 skewed feature 중 target 예측에 어느 정도 영향을 주는 feature인 `lead_desc_length` feature에만 `np.log1p()` transformation을 적용하였음.

**Basic rfc params**
```python
rfc_params_05 = {
    'n_estimators': 100, 
    'criterion': 'gini', 
    'max_depth': None, 
    'min_samples_split': 2, 
    'min_samples_leaf': 1, 
    'min_weight_fraction_leaf': 0.0, 
    'max_features': None,
    'max_leaf_nodes': None, 
    'min_impurity_decrease': 0.0, 
    'bootstrap': True, 
    'oob_score': True, 
    'n_jobs': None, 
    'random_state': hparams['seed'], 
    'verbose': 1, 
    'warm_start': False, 
    'class_weight': "balanced_subsample", 
    'ccp_alpha': 0.0004,
    'max_samples': None, 
}   
```

**Test f1 score**
- Previous best test f1 score: 0.717894

    |rfc_params|test f1 score| description |
    |:-:|:-:|:-:|
    | 05 | 0.717894 (+0.00) | params05 세팅 + `lead_desc_length`에 log transformation 적용 |

**Analysis**
- 변환 전/후를 비교했을 때 skew 정도가 완화되긴 하였음.
- 다만 target 예측에 있어 `customer_idx` feature가 미치는 영향이 너무 컸던 탓인지, transformation에 따른 성능 향상은 전혀 없었음.
- 학습에 활용가능한 28개의 feature 중 특정 feature에 대한 의존도가 너무 높은 것 같아서,, 유의미한 새로운 feature를 만들던가 모델 클래스를 바꾸어보던가 해야할 것 같음.

**Future works**
- Feature engineering 등
---
---

### 정리

`+`
- Increase number of estimators
- Decrease ccp_alpha value
---
`-`
- Use small number of features (e.g., sqrt, log_2)
- Increase ccp_alpha value
---
---

### Future works (`02_use_sampler.ipynb`와 동일)

- 모델 (e.g., svm, gbm, AdaBoost, lightGBM, XGBoost, CatBoost) 변경
- 새로운 기법 탐색 (e.g., Logistic Regression, Out-Of-Distribution Detection)
- 결측값 처리
- feature engineering
- data augmentation